**Sales Fact**

In [0]:
%sql
INSERT OVERWRITE vsqproject.gold.fact_sales
SELECT
  CAST(date_format(s.sale_date, 'yyyyMMdd') AS INT) AS DateKey,
  s.product_id AS ProductKey,
  s.customer_id AS CustomerKey,
  s.store_id AS StoreKey,
  s.quantity AS Quantity,
  s.sale_amount AS NetSales,
  0.0 AS DiscountAmount,
  (s.sale_amount - (p.price * 0.7 * s.quantity)) AS Profit
FROM vsqproject.silver.silver_sales s
JOIN vsqproject.silver.products_silver p ON s.product_id = p.product_id;

**Product Dimension**

In [0]:
%sql
-- Step 1: Create a staging view with transformed attributes
CREATE OR REPLACE TEMP VIEW product_stage AS
SELECT DISTINCT 
  product_id AS ProductKey,
  productname AS ProductName,
  category AS Category,
  brand AS Brand,
  price AS UnitPrice
FROM vsqproject.silver.products_silver;

In [0]:
%sql
--Step 2: Updating the records that have updates
MERGE INTO vsqproject.gold.dim_product AS tgt
USING product_stage AS src
ON tgt.ProductKey = src.ProductKey AND tgt.IsCurrent = 1

--  Expire old record if something changed
WHEN MATCHED AND (
    tgt.ProductName <> src.ProductName OR
    tgt.Category    <> src.Category OR
    tgt.Brand       <> src.Brand OR
    tgt.UnitPrice   <> src.UnitPrice
) THEN UPDATE SET 
    tgt.EndDate   = current_date() - INTERVAL 1 DAY,
    tgt.IsCurrent = 0;

In [0]:
%sql
--Step 3: Inserting the new records and updated records
MERGE INTO vsqproject.gold.dim_product AS tgt
USING product_stage AS src
ON tgt.ProductKey = src.ProductKey AND tgt.IsCurrent = 1
WHEN NOT MATCHED THEN 
  INSERT (
    ProductKey, ProductName, Category, Brand, UnitPrice, EffectiveDate, EndDate, IsCurrent
  )
  VALUES (
    src.ProductKey, src.ProductName, src.Category, src.Brand, src.UnitPrice, current_date(), DATE'9999-12-31', 1);

**Customer Dimension**

In [0]:
%sql
-- Step 1: Create a staging view with transformed attributes
CREATE OR REPLACE TEMP VIEW customer_stage AS
SELECT DISTINCT
  customer_id AS CustomerKey,
  CONCAT(firstname, ' ', lastname) AS FullName,
  gender AS Gender,
  CASE 
    WHEN FLOOR(DATEDIFF(current_date(), dob)/365) BETWEEN 18 AND 25 THEN '18-25'
    WHEN FLOOR(DATEDIFF(current_date(), dob)/365) BETWEEN 26 AND 35 THEN '26-35'
    WHEN FLOOR(DATEDIFF(current_date(), dob)/365) BETWEEN 36 AND 50 THEN '36-50'
    ELSE '50+'
  END AS AgeGroup,
  city,
  state
FROM vsqproject.silver.customers_silver;

In [0]:
%sql
--Step 2: Updating the records that have updates
MERGE INTO vsqproject.gold.dim_customer AS tgt
USING customer_stage AS src
ON tgt.CustomerKey = src.CustomerKey AND tgt.IsCurrent = 1
WHEN MATCHED AND (
    tgt.FullName <> src.FullName OR
    tgt.Gender   <> src.Gender OR
    tgt.AgeGroup <> src.AgeGroup OR
    tgt.city     <> src.city OR
    tgt.state    <> src.state
) THEN
  UPDATE SET
    tgt.IsCurrent = 0,
    tgt.EndDate   = current_date() - INTERVAL 1 DAY;

In [0]:
%sql
--Step 3: Inserting the new records and updated records
MERGE INTO vsqproject.gold.dim_customer AS tgt
USING customer_stage AS src
ON tgt.CustomerKey = src.CustomerKey AND tgt.IsCurrent = 1
WHEN NOT MATCHED THEN
  INSERT (CustomerKey, FullName, Gender, AgeGroup, city, state,
          EffectiveDate, EndDate, IsCurrent)
  VALUES (src.CustomerKey, src.FullName, src.Gender, src.AgeGroup,
          src.city, src.state, current_date(), DATE'9999-12-31', 1);

**Store Dimension**

In [0]:
%sql
INSERT OVERWRITE vsqproject.gold.dim_store
SELECT DISTINCT
  store_id AS StoreKey,
  store_name AS StoreName,
  city,
  state
FROM vsqproject.silver.stores;